In [1]:
using ITensors
using StatsBase
using Statistics
using ProgressMeter
using JLD
using Dates

global const nsweeps::Int = 100
global const maxdim::Int = 1000
global const cutoff::Float64 = 1e-9
global const noise::Vector{Float64} = noise=[1e-4, 1e-5, 1e-6, 0.0]

global const observer::DMRGObserver{Float64} = DMRGObserver(energy_tol=1e-6)

global const rho::Float64 = (sqrt(5) - 1) / 2  # 1 / phi
global const rho2::Float64 = (3 - sqrt(5)) / 2;  # 1 / phi^2

In [2]:
function build_HAos(Ad::Matrix{Int64}, s::Int, t::Int, A=50)
    n = size(Ad)[1]

    edges = Tuple.(findall(Ad .!= 0))
    N = size(edges)[1]

    os = OpSum()

    J = zeros(N, N)
    K = zeros(N)

    for i in 1:n
        si = findfirst(edges .== ((s, i),))
        is = findfirst(edges .== ((i, s),))

        ti = findfirst(edges .== ((t, i),))
        it = findfirst(edges .== ((i, t),))

        for j in 1:n
            sj = findfirst(edges .== ((s, j),))
            js = findfirst(edges .== ((j, s),))

            # build Hs
            if (sj != nothing) && (si != nothing) && (sj != si)
                os .+= 1, "Z", sj, "Z", si
                J[sj, si] += 1
            end

            if (js != nothing) && (is != nothing) && (js != is)
                os .+= 1, "Z", js, "Z", is
                J[js, is] += 1
            end

            if (js != nothing) && (si != nothing) && (js != si)
                os .+= -1, "Z", js, "Z", si
                J[js, si] += -1
            end

            if (sj != nothing) && (is != nothing) && (sj != is)
                os .+= -1, "Z", sj, "Z", is
                J[sj, is] += -1
            end

            tj = findfirst(edges .== ((t, j),))
            jt = findfirst(edges .== ((j, t),))

            # build Ht
            if (tj != nothing) && (ti != nothing) && (tj != ti)
                os .+= 1, "Z", tj, "Z", ti
                J[tj, ti] += +1
            end

            if (jt != nothing) && (it != nothing) && (jt != it)
                os .+= 1, "Z", jt, "Z", it
                J[jt, it] += +1
            end

            if (jt != nothing) && (ti != nothing) && (jt != ti)
                os .+= -1, "Z", jt, "Z", ti
                J[jt, ti] += -1
            end

            if (tj != nothing) && (it != nothing) && (tj != it)
                os .+= -1, "Z", tj, "Z", it
                J[tj, it] += -1
            end

            # build Hij
            if (i != s) && (i != t)
                if i == j
                    continue
                end

                ij = findfirst(edges .== ((i, j),))
                ji = findfirst(edges .== ((j, i),))
                for k in 1:n
                    if i == k
                        continue
                    end
                    ki = findfirst(edges .== ((k, i),))
                    ik = findfirst(edges .== ((i, k),))

                    if (ij != nothing) && (ik != nothing) && (ij != ik)
                        os .+= 1, "Z", ij, "Z", ik
                        J[ij, ik] += 1
                    end

                    if (ji != nothing) && (ki != nothing) && (ji != ki)
                        os .+= 1, "Z", ji, "Z", ki
                        J[ji, ki] += 1
                    end

                    if (ji != nothing) && (ik != nothing) && (ji != ik)
                        os .+= -1, "Z", ji, "Z", ik
                        J[ji, ik] += -1
                    end

                    if (ij != nothing) && (ki != nothing) && (ij != ki)
                        os .+= -1, "Z", ij, "Z", ki
                        J[ij, ki] += -1
                    end
                end
            end
        end

        # Hs
        if (si != nothing)
            os .+= -4, "Z", si
            K[si] += -4
        end
        if (is != nothing)
            os .+= +4, "Z", is
            K[is] += +4
        end
        # Ht
        if (ti != nothing)
            os .+= +4, "Z", ti
            K[ti] += +4
        end
        if (it != nothing)
            os .+= -4, "Z", it
            K[it] += -4
        end
    end

    J = J .+ transpose(J)

    os = OpSum()
    for i in 1:N
        for j in i+1:N
            if J[i, j] != 0
                os += J[i, j], "Z", i, "Z", j
            end
        end
    end

    for i in 1:N
        if K[i] != 0
            os += K[i], "Z", i
        end
    end

    os *= A / 4
    J *= A / 4
    K *= A / 4
    return os, J, K
end


function build_HBos(N::Int, W, B=1)
    os = OpSum()
    for j in 1:N
        os .+= B/2*W[j], "Z", j
    end

    return os
end


function build_Hxos(N::Int)
    os = OpSum()
    for j in 1:N
        os .+= -1, "X", j
    end

    return os
end;

In [3]:
function compute_delta(sites::Vector{Index{Int64}}, N::Int, s::Float64, HA::MPO, HB::MPO, Hx::MPO;
                       psi0_init=MPS()::MPS, psi1_init=MPS()::MPS)
    H = (1-s)*Hx + s*(HA + HB)

    if psi0_init.rlim == 0
        state = StatsBase.sample(["Up", "Dn"], N)
        psi0_init = MPS(sites, state)
    end

    E0, psi0 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                    eigsolve_krylovdim=5, outputlevel=0, observer=observer)

    if psi1_init.rlim == 0
        state = StatsBase.sample(["Up", "Dn"], N)
        psi1_init = MPS(sites, state)
    end

    E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                    eigsolve_krylovdim=6, outputlevel=0, observer=observer, weight=2)

    if E1 > E0
        return E1-E0, psi0, psi1
    else
        return E0-E1, psi1, psi0
    end
end


function find_delta_min(sites::Vector{Index{Int64}}, N::Int, p0::Vector{Union{Nothing, Int64}}, p1::Vector{Union{Nothing, Int64}}, HA::MPO, HB::MPO, Hx::MPO; tol=1e-4::Float64)
    a = 0
    b = 0.6

    h = b - a
    c = a + rho2 * h
    d = a + rho * h

    # Required steps to achieve tolerance
    n = trunc(Int, ceil(log(tol / h) / log(rho)))

    ground_state = ["Dn" for i in 1:N]
    for e in p0
        ground_state[e] = "Up"
    end

    excited_state = ["Dn" for i in 1:N]
    for e in p1
        excited_state[e] = "Up"
    end

    psi0 = MPS(sites, ground_state)
    psi1 = MPS(sites, excited_state)

    # println(Dates.format(now(), "HH:MM"))
    _, psi0, psi1 = compute_delta(sites, N, 0.5, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)

    yd, psi0, psi1 = compute_delta(sites, N, d, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
    yc, psi0, psi1 = compute_delta(sites, N, c, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)

    for k in 1:n
        if yc < yd
            b = d
            d = c
            yd = yc
            h = rho * h
            c = a + rho2 * h

            yc, psi0, psi1 = compute_delta(sites, N, c, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        else
            a = c
            c = d
            yc = yd
            h = rho * h
            d = a + rho * h

            yd, psi0, psi1 = compute_delta(sites, N, d, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        end
    end

    if yc < yd
        sc = (a + d) / 2
    else
        sc = (b + c) / 2
    end

    dmin, psi0, psi1 = compute_delta(sites, N, sc, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)

    return sc, dmin
end;

In [4]:
ITensors.Strided.disable_threads()
ITensors.Strided.disable_threaded_mul()

ITensors.enable_combine_contract()
ITensors.enable_contraction_sequence_optimization()

In [5]:
using PyCall

py"""
from itertools import islice

import networkx as nx
from networkx.generators.harary_graph import hkn_harary_graph

def two_sp(n, s, t, edges, W):
    G = hkn_harary_graph(3, n)
    G = nx.DiGraph(G)

    for i, e in enumerate(edges):
        G[e[0]][e[1]]['weight'] = W[i]

    p0, p1 = list(islice(nx.shortest_simple_paths(G, s, t, weight="weight"), 2))
    
    return p0, p1
"""

In [7]:
n = 9

graph_file = "analysis/data/harary/graph_n=$n.jld"

Ad = load(graph_file, "Ad")
source = load(graph_file, "source")
target = load(graph_file, "target")

edges = Tuple.(findall(Ad .!= 0))
N = size(edges)[1]
sites = siteinds("S=1/2", N)

A = N
B = 1

os_HA, J, K = build_HAos(Ad, source, target, A)
os_Hx = build_Hxos(N)

HA = MPO(os_HA, sites)
Hx = MPO(os_Hx, sites)

py_edges = [(e[1]-1, e[2]-1) for e in edges]
    
W = zeros(N)
for i in 1:N
    if reverse(edges[i]) in edges[1:i-1]
        W[i] = W[findfirst(edges .== (reverse(edges[i]),))]
    else
        W[i] = randn() ./ 6 .+ 0.5
    end
end
W[W .> 1] .= 1
W[W .< 0] .= 0
        
os_HB = build_HBos(N, W, B)
HB = MPO(os_HB, sites)

p0, p1 = py"two_sp"(n, source-1, target-1, py_edges, W)
        
p0 .+= 1
p1 .+= 1

p0 = [(p0[i], p0[i+1]) for i in 1:(length(p0)-1)]
p1 = [(p1[i], p1[i+1]) for i in 1:(length(p1)-1)]

p0 = indexin(p0, edges)
p1 = indexin(p1, edges)

ground_state = ["Dn" for i in 1:N]
for e in p0
    ground_state[e] = "Up"
end

excited_state = ["Dn" for i in 1:N]
for e in p1
    excited_state[e] = "Up"
end

psi0_init = MPS(sites, ground_state)
psi1_init = MPS(sites, excited_state)

s = 0.5
H = (1-s)*Hx + s*(HA + HB)

@time E0, psi0 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0_init; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=3, outputlevel=0, observer=observer)
println(E0)

@time E0, psi0 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0_init; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=4, outputlevel=0, observer=observer)
println(E0)

@time E0, psi0 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0_init; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=5, outputlevel=0, observer=observer)
println(E0)

@time E0, psi0 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0_init; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=6, outputlevel=0, observer=observer)
println(E0)

println("\n")

@time E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=4, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=5, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=6, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=7, outputlevel=0, observer=observer, weight=2)
println(E1)

println("\n")

s = 0.3
H = (1-s)*Hx + s*(HA + HB)

@time E0, psi02 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=3, outputlevel=0, observer=observer)
println(E0)

@time E0, psi02 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=4, outputlevel=0, observer=observer)
println(E0)

@time E0, psi02 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=5, outputlevel=0, observer=observer)
println(E0)

@time E0, psi02 = dmrg([(1-s)*Hx,s*HA,s*HB], psi0; nsweeps=nsweeps, maxdim=maxdim,
                cutoff=cutoff, noise=noise, eigsolve_krylovdim=6, outputlevel=0, observer=observer)
println(E0)

println("\n")

@time E1, psi12 = dmrg(H, [psi02], psi1; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=4, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi12 = dmrg(H, [psi02], psi1; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=5, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi12 = dmrg(H, [psi02], psi1; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=6, outputlevel=0, observer=observer, weight=2)
println(E1)

@time E1, psi12 = dmrg(H, [psi02], psi1; nsweeps=nsweeps, maxdim=maxdim, cutoff=cutoff, noise=noise,
                eigsolve_krylovdim=7, outputlevel=0, observer=observer, weight=2)
println(E1)

  0.879312 seconds (5.01 M allocations: 1.582 GiB, 16.49% gc time)
-227.19944911514324
  0.671425 seconds (3.19 M allocations: 1.263 GiB, 15.02% gc time)
-227.19946424662075
  0.552830 seconds (3.11 M allocations: 1.259 GiB, 17.43% gc time)
-227.19946437558357
  0.509315 seconds (2.78 M allocations: 1.216 GiB, 17.39% gc time)
-227.19947036027384


  0.390542 seconds (2.13 M allocations: 804.496 MiB, 16.84% gc time)
-227.1310473307883
  0.337650 seconds (2.03 M allocations: 728.445 MiB, 16.73% gc time)
-227.1310482197701
  0.358811 seconds (2.09 M allocations: 771.477 MiB, 18.14% gc time)
-227.13107346569296
  0.502338 seconds (2.84 M allocations: 1.077 GiB, 17.65% gc time)
-227.13110543383493


  9.523158 seconds (5.84 M allocations: 42.406 GiB, 11.92% gc time)
-137.09971835374404
  8.425411 seconds (4.17 M allocations: 43.768 GiB, 9.05% gc time)
-137.09972398331618
  9.313417 seconds (3.53 M allocations: 45.785 GiB, 7.13% gc time)
-137.0997253600861
  9.065142 seconds (3.27 M allocati